In [132]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import sympy as sp
import control as ct
import scienceplots
import latex
plt.style.use('_mpl-gallery')

In [133]:
# Parâmetros do sistema
m0  = 1
m1  = 1
m2  = 1
L1  = 0.1
L2  = 0.1
l1  = L1 / 2
l2  = L2 / 2
g   = 9.81
J1  = 0.00083
J2  = 0.00083
c0  = 0.01
c1  = 0.007
c2  = 0.007

# Simplificações
h1 = m0 + m1 + m2
h2 = (m1 + 2 * m2) * l1
h3 = m2 * l2
h4 = J1 + m1 * l1**2 + 4 * m2 * l1**2
h5 = 2 * l1 * l2 * m2
h6 = J2 + 4 * m2 * l2**2

w1 = g * l1 * (m1 + 2 * m2)
w2 = g * l2 * m2

In [134]:
# Variáveis simbólicas
Theta0, Theta1, Theta2, Phi1 ,Phi2= sp.symbols('Theta0 Theta1 Theta2 Phi1 Phi2')
DTheta0, DTheta1, DTheta2 = sp.symbols('DTheta0 DTheta1 DTheta2')
u = sp.symbols('u')
x_ = sp.Matrix([Theta0, Theta1, Theta2, DTheta0, DTheta1, DTheta2])
x_1 = sp.Matrix([Theta0,Phi1,Phi2,DTheta0, DTheta1, DTheta2])

# Matrizes simbólicas na dinâmica não linear
D = sp.Matrix([
    [h1, h2 * sp.cos(Theta1), h3 * sp.cos(Theta2)],
    [h2 * sp.cos(Theta1), h4, h5 * sp.cos(Theta1 - Theta2)],
    [h3 * sp.cos(Theta2), h5 * sp.cos(Theta1 - Theta2), h6]
])

C = sp.Matrix([
    [c0, -2*h2 * sp.sin(Theta1) * DTheta1, -2*h3 * sp.sin(Theta2) * DTheta2],
    [0, c1 + c2, -c2 + 2*h5 * sp.sin(Theta1 - Theta2) * DTheta2],
    [0, -c2 + 2*h5 * sp.sin(Theta1 - Theta2) * DTheta1, c2]
])

G = sp.Matrix([
    [0],
    [-w1 * sp.sin(Theta1)],
    [-w2 * sp.sin(Theta2)]
])

H = sp.Matrix([1, 0, 0])

In [135]:
# Matrizes do sistema não linear
Anl = sp.Matrix.hstack(sp.zeros(3, 3), sp.eye(3))
Anl = sp.Matrix.vstack(Anl, sp.Matrix.hstack(sp.zeros(3, 3), -D.inv() * C))

Bnl = sp.Matrix.vstack(sp.zeros(3, 1), D.inv() * H)

Hnl = sp.Matrix.vstack(sp.zeros(3, 1), -D.inv() * G)

In [136]:
# Dinâmica não linear completa do sistema
nonLeqn = Anl * x_ + Hnl + Bnl * u
nonLeqn

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [138]:
A_1 = nonLeqn.subs({sp.sin(Theta1):Phi1, sp.sin(Theta2):Phi2, sp.cos(Theta1):1,sp.cos(Theta1-Theta2):1,sp.cos(Theta2):1, u:0})
A_1


Matrix([
[                                                                                                                                                                                                                                                                                                       DTheta0],
[                                                                                                                                                                                                                                                                                                       DTheta1],
[                                                                                                                                                                                                                                                                                                       DTheta2],
[-0.0076470369660911*DTheta0 + DTheta1*(0.229411108982733*DTheta1*Phi1 - 

In [139]:
A_2 = A_1.jacobian(x_1)
A_2

Matrix([
[0,                                               0,                                                0,                    1,                                                                                                                                                                                 0,                                                                                                                                                                                  0],
[0,                                               0,                                                0,                    0,                                                                                                                                                                                 1,                                                                                                                                                                                  0],
[0,                  

In [140]:
A_3 = A_2.subs({Theta0: 0, Theta1: 0, Theta2: 0, DTheta0: 0, DTheta1: 0, DTheta2: 0})
A_3

Matrix([
[0,                 0,                0,                    1,                 0,                   0],
[0,                 0,                0,                    0,                 1,                   0],
[0,                 0,                0,                    0,                 0,                   1],
[0, -12.9576188227818, 0.26238903157567,  -0.0076470369660911, 0.127024691255205, -0.0653846424889985],
[0,   282.72025354327, -23.567877087635,   0.0880572125231515, -3.02616987322196,    1.68125531338309],
[0,  -70.703631262905, 54.9602893683649, -0.00534941960398918,  1.45702814435361,   -1.12068739080938]])

In [141]:
# Linearização do sistema não linear usando o método Jacobiano
Aa = nonLeqn.subs({Theta0: 0, Theta1: 0, Theta2: 0, DTheta0: 0, DTheta1: 0, DTheta2: 0, sp.sin(Theta1):Phi1, sp.sin(Theta2):Phi2, sp.cos(Theta1):1,sp.cos(Theta2):1})
Aaa = Aa.jacobian(x_1)
#Ass = Aa.subs({Theta0: 0, Theta1: 0, Theta2: 0, DTheta0: 0, DTheta1: 0, DTheta2: 0, sp.sin(Theta1):Phi1, sp.sin(Theta2):Phi2, sp.cos(Theta1):1,sp.cos(Theta2):1})
#Ass = Aaa.subs({Theta0: 0, Theta1: 0, Theta2: 0, DTheta0: 0, DTheta1: 0, DTheta2: 0})
Aaa

Matrix([
[0,                 0,                0, 0, 0, 0],
[0,                 0,                0, 0, 0, 0],
[0,                 0,                0, 0, 0, 0],
[0, -12.9576188227818, 0.26238903157567, 0, 0, 0],
[0,   282.72025354327, -23.567877087635, 0, 0, 0],
[0, -70.7036312629051, 54.9602893683649, 0, 0, 0]])

In [ ]:

Bb = nonLeqn.jacobian([u])
Bss = Bb.subs({Theta0: 0, Theta1: 0, Theta2: 0, DTheta0: 0, DTheta1: 0, DTheta2: 0})
Bss

In [ ]:
# Controlador LQR
Q = np.diag([50, 90, 90, 0, 0, 0])
R = 0.1

# Função para calcular K, S, P utilizando scipy
[K, S, P] = ct.lqr(A_3,Bss,Q,R)

# Representação dos polos
reale = np.real(P)
immaginario = np.imag(P)

# Gráfico dos polos no plano complexo
plt.figure()
plt.plot(reale, immaginario, 'rx', markersize=10)
plt.xlabel('Parte Real')
plt.ylabel('Parte Imaginária')
plt.title('Diagrama de Pólos (Cartesiano)')
plt.grid(True)
plt.axis('equal')
plt.show()


In [ ]:
# Simular o sistema linear para condições iniciais
Al = A_3
Bl = Bss
Cl = np.eye(6)
Dl = 0
sys = ct.ss(Al,Bl,Cl,Dl)

t_ma = np.linspace(0, 5, 501)
x0_ma =  [0, np.deg2rad(2), -np.deg2rad(2), 0, 0, 0]

t_ma,y_ma = ct.initial_response(sys, t_ma, x0_ma)


In [ ]:
plt.style.use('_mpl-gallery')
plt.figure(figsize=(8,8))
# Plot the the input delta(t) and the outputs: psi(t) and Y(t)
plt.subplot(3,1,1)
plt.plot(t_ma,y_ma[0],'b',linewidth=2,label='Trajetória Referência')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('Posição [m]',fontsize=11)
plt.grid(True)
plt.legend(loc='upper right',fontsize='small')

plt.subplot(3,1,2)
plt.plot(t_ma,y_ma[1]*180/np.pi,'b',linewidth=2,label='Ângulo do pêndulo inferior_ref')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('phi1-posição [rad]',fontsize=11)
plt.grid(True)
plt.legend(loc='center right',fontsize='small')

plt.subplot(3,1,3)
plt.plot(t_ma,y_ma[2]*180/np.pi,'b',linewidth=2,label='pêndulo superior_ref')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('phi2-posição [rad]',fontsize=11)
plt.grid(True)
plt.legend(loc='center right',fontsize='small')

plt.show()

In [ ]:
# Fechando a malha:
A_mf = Al - Bl@K
B_mf = Bl
C_mf = Cl
D_mf = Dl
sys_mf = ct.ss(A_mf, B_mf, C_mf, D_mf)

t = np.linspace(0, 5, 501)
x0 =  [0, np.deg2rad(2), -np.deg2rad(2), 0, 0, 0]

t,y = ct.initial_response(sys_mf, t, x0)

In [ ]:

plt.style.use('_mpl-gallery')
plt.figure(figsize=(8,8))
# Plot the the input delta(t) and the outputs: psi(t) and Y(t)
plt.subplot(3,1,1)
plt.plot(t,y[0],'b',linewidth=2,label='Trajetória Referência')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('Posição [m]',fontsize=11)
plt.grid(True)
plt.legend(loc='upper right',fontsize='small')

plt.subplot(3,1,2)
plt.plot(t,y[1]*180/np.pi,'b',linewidth=2,label='Ângulo do pêndulo inferior_ref')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('phi1-posição [rad]',fontsize=11)
plt.grid(True)
plt.legend(loc='center right',fontsize='small')

plt.subplot(3,1,3)
plt.plot(t,y[2]*180/np.pi,'b',linewidth=2,label='pêndulo superior_ref')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('phi2-posição [rad]',fontsize=11)
plt.grid(True)
plt.legend(loc='center right',fontsize='small')

plt.show()

In [ ]:
# Dinâmica não linear do sistema
def nlode(x, t, K, Anl, Bnl, Hnl):
    A = np.array(Anl.subs({Theta0: x[0], Theta1: x[1], Theta2: x[2], DTheta0: x[3], DTheta1: x[4], DTheta2: x[5]})).astype(np.float64)
    B = np.array(Bnl.subs({Theta0: x[0], Theta1: x[1], Theta2: x[2], DTheta0: x[3], DTheta1: x[4], DTheta2: x[5]})).astype(np.float64)
    H = np.array(Hnl.subs({Theta0: x[0], Theta1: x[1], Theta2: x[2], DTheta0: x[3], DTheta1: x[4], DTheta2: x[5]})).astype(np.float64)
    
    u =-K@x  # Controlador LQR
    
    dxdt = dxdt = np.dot(A, x) + H.flatten() + np.dot(B, u)
    return dxdt.flatten()

In [ ]:
# Simular modelo não linear com odeint
x0 = [0, np.deg2rad(5), -np.deg2rad(5), 0, 0, 0]
t_a = np.linspace(0, 5, 500)
x = odeint(nlode, x0, t_a, args=(K, Anl, Bnl, Hnl))

In [ ]:
# Converter de radianos para graus
x.T[1]= np.rad2deg(x.T[1])
x.T[2] = np.rad2deg(x.T[2])

In [ ]:
plt.figure(figsize=(8,8))
# Plot the the input delta(t) and the outputs: psi(t) and Y(t)
plt.subplot(3,1,1)
plt.plot(x.T[0],'b',linewidth=2,label='Trajetória Referência')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('Posição [m]',fontsize=11)
plt.grid(True)
plt.legend(loc='upper right',fontsize='small')

plt.subplot(3,1,2)
plt.plot(x.T[1],'b',linewidth=2,label='Ângulo do pêndulo inferior_ref')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('phi1-posição [rad]',fontsize=11)
plt.grid(True)
plt.legend(loc='center right',fontsize='small')

plt.subplot(3,1,3)
plt.plot(x.T[2],'b',linewidth=2,label='pêndulo superior_ref')
plt.xlabel('t-tempo [s]',fontsize=11)
plt.ylabel('phi2-posição [rad]',fontsize=11)
plt.grid(True)
plt.legend(loc='center right',fontsize='small')

plt.show()